In [1]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 5.1 MB/s 
     |████████████████████████████████| 9.3 MB 3.0 MB/s 
     |████████████████████████████████| 1.6 MB 45.2 MB/s 
     |████████████████████████████████| 108 kB 34.7 MB/s 


In [2]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [4]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [5]:
df.drop(['packer_type'],axis=1,inplace=True)

In [6]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,256,4,1,0,1,...,1,1,3,0,6.603616,5.443362,1181520,6.627552,1,0
1,144,3,4,65535,184,184,4,1,0,1,...,1,1,3,0,5.205926,2.123522,7680,5.318221,0,0
2,144,3,4,65535,184,272,5,1,0,1,...,1,1,4,0,6.238000,3.380859,57872,6.507758,1,0
3,144,3,4,65535,184,184,1,1,0,1,...,1,0,1,0,0.000000,0.000000,95616,4.575092,1,0
4,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,6.355626,0.702621,48128,5.545531,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,216,5,1,1,1,...,1,0,5,0,6.174602,3.155928,365568,7.546568,0,1
5206,80,2,4,65535,184,256,7,1,0,1,...,1,4,3,0,0.000000,0.000000,98816,6.947195,0,1
5207,144,3,4,65535,184,216,5,1,0,1,...,1,2,3,0,6.503422,3.790871,227328,7.823114,0,1
5208,144,3,4,65535,184,248,5,1,1,1,...,1,1,4,0,6.115208,7.919091,271616,7.886012,0,1


In [7]:
from sklearn.utils import shuffle
df = shuffle(df)

In [8]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
4203,80,2,4,65535,184,256,3,1,1,1,...,1,2,1,1,0.000000,0.000000,106496,7.545731,1,1
4043,144,3,4,65535,184,240,4,1,1,1,...,1,0,4,0,6.493571,1.797418,59656,5.997324,1,1
123,144,3,4,65535,184,240,6,1,0,1,...,1,2,4,0,6.336881,1.367143,64440,6.536607,1,0
3598,144,3,4,65535,184,224,4,1,1,1,...,1,0,4,0,6.647329,3.989093,122880,7.362105,0,1
1523,144,3,4,65535,184,344,5,1,0,1,...,1,1,4,0,6.553739,5.927938,627216,5.427818,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,0,6.232950,5.541258,25088,6.158367,1,0
1145,144,3,4,65535,184,248,4,1,0,1,...,1,1,3,0,6.505196,0.839276,143888,6.524974,0,0
4171,144,3,4,65535,184,128,4,1,0,1,...,1,0,4,0,7.827974,0.000000,33792,6.487991,0,1
1132,144,3,4,65535,184,192,4,1,0,1,...,1,1,3,0,3.172938,0.000000,228152,3.481932,0,0


In [9]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [10]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [11]:
df.drop(['index'],axis=1,inplace=True)

In [12]:
df_5percent=df[0:1302] 


In [13]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,80,2,4,65535,184,256,3,1,1,1,...,1,2,1,1,0.000000,0.000000,106496,7.545731,1,1
1,144,3,4,65535,184,240,4,1,1,1,...,1,0,4,0,6.493571,1.797418,59656,5.997324,1,1
2,144,3,4,65535,184,240,6,1,0,1,...,1,2,4,0,6.336881,1.367143,64440,6.536607,1,0
3,144,3,4,65535,184,224,4,1,1,1,...,1,0,4,0,6.647329,3.989093,122880,7.362105,0,1
4,144,3,4,65535,184,344,5,1,0,1,...,1,1,4,0,6.553739,5.927938,627216,5.427818,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,144,3,4,65535,184,256,4,1,1,1,...,1,0,4,0,6.595372,4.641918,11490688,7.957220,1,0
1298,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,6.162536,1.427149,498176,6.144838,1,0
1299,144,3,4,65535,184,272,5,1,0,1,...,1,1,4,0,6.470026,5.045581,407160,6.216273,1,0
1300,144,3,4,65535,184,280,3,1,1,1,...,1,0,3,0,6.350259,2.888890,67144,6.516870,1,0


In [14]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (642, 69)
class 1: (660, 69)


In [15]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [16]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0       80.0   2.0        4.0     65535.0  184.0     256.0               3.0   
1      144.0   3.0        4.0     65535.0  184.0     240.0               4.0   
2      144.0   3.0        4.0     65535.0  184.0     240.0               6.0   
3      144.0   3.0        4.0     65535.0  184.0     224.0               4.0   
4      144.0   3.0        4.0     65535.0  184.0     344.0               5.0   
...      ...   ...        ...         ...    ...       ...               ...   
1297   144.0   3.0        4.0     65535.0  184.0     256.0               4.0   
1298   144.0   3.0        4.0     65535.0  184.0     224.0               5.0   
1299   144.0   3.0        4.0     65535.0  184.0     272.0               5.0   
1300   144.0   3.0        4.0     65535.0  184.0     280.0               3.0   
1301   144.0   3.0        4.0     65535.0  184.0     232.0               5.0   

      C

In [17]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,1302.000000,1302.000000,1302.00000,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,1302.0,...,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,1302.000000,1.302000e+03,1302.000000,1302.000000,1302.000000
mean,138.231951,2.902458,3.97235,65145.955453,183.259601,223.188940,4.730415,0.991551,0.353303,1.0,...,0.997696,1.402458,3.327957,0.150538,4.956873,2.445519,9.293809e+05,6.352814,0.533026,0.506912
std,20.044799,0.390719,0.33154,4971.151843,18.372988,49.064606,2.028311,0.091562,0.478179,0.0,...,0.047965,1.761994,1.127861,0.357735,2.504247,2.608012,6.665404e+06,1.113271,0.499100,0.500144
min,0.000000,0.000000,0.00000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.939626,0.000000,0.000000
25%,144.000000,3.000000,4.00000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,4.576729,0.000000,6.489800e+04,5.682854,0.000000,0.000000
50%,144.000000,3.000000,4.00000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.156160,1.581947,1.218560e+05,6.398364,1.000000,1.000000
75%,144.000000,3.000000,4.00000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.493571,4.507766,3.109120e+05,7.220176,1.000000,1.000000
max,144.000000,8.000000,4.00000,65535.000000,332.000000,600.000000,34.000000,1.000000,1.000000,1.0,...,1.000000,31.000000,8.000000,1.000000,7.999070,7.999532,1.223571e+08,7.999997,1.000000,1.000000


In [18]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [19]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [20]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [21]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
1.0      52.5
0.0      47.5
dtype: float64

In [22]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
0.0      52.5
1.0      47.5
dtype: float64

In [23]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [24]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [25]:
circuit(weights_init, data[0])

tensor(0.09618608, requires_grad=True)

In [26]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias


In [27]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss


In [28]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [29]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [30]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.31516863, requires_grad=False), tensor(-0.2078869, requires_grad=False)], Y = -1
X = [tensor(-0.21349526, requires_grad=False), tensor(0.01736999, requires_grad=False)], Y = -1
X = [tensor(-0.33370508, requires_grad=False), tensor(0.35714234, requires_grad=False)], Y = -1
X = [tensor(-0.32586346, requires_grad=False), tensor(-0.21030139, requires_grad=False)], Y = -1
X = [tensor(-0.21083278, requires_grad=False), tensor(0.01791442, requires_grad=False)], Y =  1


In [31]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [32]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.1366465 | Accuracy: 0.5000000 
Iter:     2 | Cost: 1.0053555 | Accuracy: 0.5000000 
Iter:     3 | Cost: 1.0442238 | Accuracy: 0.4937500 
Iter:     4 | Cost: 1.3299686 | Accuracy: 0.4875000 
Iter:     5 | Cost: 1.4586211 | Accuracy: 0.4875000 
Iter:     6 | Cost: 1.1675720 | Accuracy: 0.4937500 
Iter:     7 | Cost: 0.9975459 | Accuracy: 0.5000000 
New best
Iter:     8 | Cost: 0.9954872 | Accuracy: 0.5125000 
Iter:     9 | Cost: 1.0054251 | Accuracy: 0.5125000 
New best
Iter:    10 | Cost: 0.9983583 | Accuracy: 0.5187500 
Iter:    11 | Cost: 0.9977463 | Accuracy: 0.5062500 
Iter:    12 | Cost: 1.0043866 | Accuracy: 0.5062500 
Iter:    13 | Cost: 0.9998777 | Accuracy: 0.5000000 
Iter:    14 | Cost: 1.0105918 | Accuracy: 0.5000000 
Iter:    15 | Cost: 1.0305045 | Accuracy: 0.5000000 
Iter:    16 | Cost: 1.0169606 | Accuracy: 0.5000000 
Iter:    17 | Cost: 0.9939194 | Accuracy: 0.5000000 
Iter:    18 | Cost: 0.9825294 | Accuracy: 0.5125000 
Iter:    19 | Cost:

In [33]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [34]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 0.9864383824359015, Accuracy: 48.0%


In [35]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,-1.0,1.0
1,1.0,-1.0
2,-1.0,-1.0
3,-1.0,-1.0
4,1.0,-1.0
5,-1.0,-1.0
6,-1.0,1.0
7,-1.0,1.0
8,-1.0,-1.0
9,-1.0,-1.0


In [36]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.76      0.50      0.60        32
         1.0       0.16      0.38      0.22         8

    accuracy                           0.48        40
   macro avg       0.46      0.44      0.41        40
weighted avg       0.64      0.47      0.53        40

0.15789473684210525
0.375
0.22222222222222218
0.4375
